<h1>目录<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读入数据集" data-toc-modified-id="读入数据集-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读入数据集</a></span></li><li><span><a href="#构建用户和物品之间的关系矩阵" data-toc-modified-id="构建用户和物品之间的关系矩阵-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>构建用户和物品之间的关系矩阵</a></span><ul class="toc-item"><li><span><a href="#将矩阵中的空值填充成0" data-toc-modified-id="将矩阵中的空值填充成0-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>将矩阵中的空值填充成0</a></span></li></ul></li><li><span><a href="#计算用户相似度(余弦相似度)" data-toc-modified-id="计算用户相似度(余弦相似度)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>计算用户相似度(余弦相似度)</a></span><ul class="toc-item"><li><span><a href="#计算皮尔逊相关性" data-toc-modified-id="计算皮尔逊相关性-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>计算皮尔逊相关性</a></span></li></ul></li><li><span><a href="#构建推荐系统的推荐指数的功能" data-toc-modified-id="构建推荐系统的推荐指数的功能-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>构建推荐系统的推荐指数的功能</a></span></li><li><span><a href="#把上面的计算过程打包成函数进行使用" data-toc-modified-id="把上面的计算过程打包成函数进行使用-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>把上面的计算过程打包成函数进行使用</a></span></li><li><span><a href="#构建预测函数" data-toc-modified-id="构建预测函数-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>构建预测函数</a></span><ul class="toc-item"><li><span><a href="#把预测功能打包成函数" data-toc-modified-id="把预测功能打包成函数-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>把预测功能打包成函数</a></span></li></ul></li><li><span><a href="#构建最终的推荐功能" data-toc-modified-id="构建最终的推荐功能-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>构建最终的推荐功能</a></span><ul class="toc-item"><li><span><a href="#对上面的数据进行分组操作" data-toc-modified-id="对上面的数据进行分组操作-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>对上面的数据进行分组操作</a></span></li><li><span><a href="#用apply调用上面的函数进行分组操作" data-toc-modified-id="用apply调用上面的函数进行分组操作-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>用apply调用上面的函数进行分组操作</a></span></li><li><span><a href="#或者是直接调用写好的函数" data-toc-modified-id="或者是直接调用写好的函数-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>或者是直接调用写好的函数</a></span></li></ul></li></ul></div>

# 手写实现协同过滤算法

因为算法比较困难, 所以我们使用课件中的简单数据集先去进行模型试验.

In [1]:
import numpy as np
import pandas as pd

## 读入数据集

In [29]:
# 把课件中的example.txt 读入进来
df = pd.read_csv('ml-100k/u.data', sep='\t',header = None )
df.head()

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
# header = None 不将数据集中的第一列作为列名读取
# 给读进来的表格设置一个列名, 重新传递列名
df.columns = ['用户ID','产品ID','评分','时间']
df

,用户ID,产品ID,评分,时间
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [31]:
df.shape

(100000, 4)

In [32]:
# 将时间这列删除掉
del df['时间']

In [33]:
df.head()

,用户ID,产品ID,评分
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


## 构建用户和物品之间的关系矩阵

index=None, columns=None, values=None

index ,用哪里列作为行

columns, 用哪个作为透视表的列

values, 用那一列作为值

In [34]:
# 使用的是透视表函数
df_pivot = df.pivot(index='用户ID',columns='产品ID',values= '评分')

In [35]:
df_pivot

产品ID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
用户ID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# 数据集中一共有 943个用户 , 1682 个 电影
df_pivot.shape

(943, 1682)

### 将矩阵中的空值填充成0

In [37]:
# inplace = True 直接在原数据集中进行替换
df_pivot.fillna(0 , inplace=True)

In [38]:
df_pivot

产品ID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
用户ID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# 将上面的DataFrame 转换成数组
freq_matrix = df_pivot.values
freq_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

## 计算用户相似度(余弦相似度)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
# 计算用户余弦相似度矩阵
cosine_similarity(freq_matrix)

array([[1.        , 0.16693098, 0.04745954, ..., 0.14861694, 0.17950788,
        0.39817474],
       [0.16693098, 1.        , 0.11059132, ..., 0.16148478, 0.17226781,
        0.10579788],
       [0.04745954, 0.11059132, 1.        , ..., 0.10124256, 0.13341615,
        0.02655587],
       ...,
       [0.14861694, 0.16148478, 0.10124256, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.17950788, 0.17226781, 0.13341615, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.39817474, 0.10579788, 0.02655587, ..., 0.09511958, 0.18246466,
        1.        ]])

In [42]:
# 将用户相似度计算出来并进行保存
user_similar_matrix = cosine_similarity(freq_matrix)

### 计算皮尔逊相关性

因为corr默认计算的是列之间的相关性,而我们要计算的是行之间的相关性, 那么我们先对矩阵进行转置, 将行列进行调换, 然后再计算

In [43]:
# 皮尔孙相关系数的计算
df_pivot.T.corr()

用户ID,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
用户ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.106322,-0.016424,0.021716,0.302592,0.345976,0.320017,0.276208,0.038812,0.288491,...,0.285923,0.068878,0.189272,0.146979,0.119444,0.060175,0.247483,0.115504,0.108984,0.323231
2,0.106322,1.000000,0.082680,0.160323,0.020218,0.197844,0.022886,0.072772,0.143716,0.106861,...,0.106416,0.288550,0.325233,0.408466,0.289112,0.204094,0.190848,0.144177,0.138292,0.054235
3,-0.016424,0.082680,1.000000,0.331674,-0.028322,0.017673,-0.013384,0.055385,0.043505,0.012080,...,-0.020605,0.018746,0.124174,0.046247,0.088974,-0.001145,0.127539,0.084532,0.102025,-0.023601
4,0.021716,0.160323,0.331674,1.000000,-0.002426,0.030206,0.041056,0.171116,0.089372,0.024018,...,0.016641,0.019738,0.105135,0.179598,0.122649,0.010890,0.174904,0.140811,0.149575,0.025367
5,0.302592,0.020218,-0.028322,-0.002426,1.000000,0.153409,0.279677,0.209280,0.025496,0.117166,...,0.272866,0.039333,0.013999,0.039519,0.085548,0.024292,0.182466,0.112571,0.096603,0.248991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.060175,0.204094,-0.001145,0.010890,0.024292,0.059487,0.032591,0.068270,0.021669,0.018157,...,0.015185,0.416779,0.223173,0.207318,0.410470,1.000000,0.049517,0.164739,0.008036,0.100749
940,0.247483,0.190848,0.127539,0.174904,0.182466,0.295949,0.251988,0.213408,0.096886,0.287583,...,0.273078,0.074005,0.128224,0.152222,0.124941,0.049517,1.000000,0.122478,0.221234,0.182613
941,0.115504,0.144177,0.084532,0.140811,0.112571,0.113858,0.008168,0.129096,0.132400,0.057182,...,0.012887,0.189966,0.269370,0.221696,0.297188,0.164739,0.122478,1.000000,0.080130,0.065300


## 构建推荐系统的推荐指数的功能

## 把上面的计算过程打包成函数进行使用

In [44]:
from recommend import cal_recommend_index_by_users

In [45]:
# 这个代表 对于 用户5 ,计算产品1 对他的推荐指数
cal_recommend_index_by_users(k = 7,user_id= 5,item_id = 1, 
            freq_matrix=freq_matrix , user_similar_matrix= user_similar_matrix)

1.0

## 构建预测函数

1. 先构建一个空的矩阵, 用户填充用户和产品之间的推荐指数是
2. 构建的这个矩阵必须是和freq_matrix这个矩阵形状相同
3. 写一个两层循环, 分别对矩阵中的值进行运算, 采用刚才写好的cal_recommend_index函数进行计算其中的值.
4. 前提条件: 如果原来freq_matrix中的值已经有了,则不需要计算,如果值为0,则计算这个值,并填充到构建的空矩阵当中去

np.zeros_like: 函数功能, 构建一个0矩阵,形状和传入的矩阵相同

第一个参数a: 传入的这个矩阵

### 把预测功能打包成函数

In [46]:
def cal_recommend_index_matrix(freq_matrix, user_similar_matrix ):
    """函数功能: 传入用户评分矩阵, 计算并返回用户对所有产品的推荐矩阵
    参数:
    freq_matrix: 用户评分矩阵
    user_similar_matrix: 用户相似度矩阵
    返回值: 推荐系数矩阵"""
    predict_matrix = np.zeros_like(freq_matrix)

    # 写一个两层循环, 分别对predict_matrix 矩阵中的每一个位置的值进行遍历
    # predict_matrix.shape[0] 代表用户个数 , range()生成一个线性序列和这个长度相同
    for user_id in range(predict_matrix.shape[0]):
        print("user_id: ",user_id)
        # 对每行中的每一个值进行遍历
        for item_id in range(predict_matrix.shape[1]):
            # 首先判断原来这个用户有没有对这个产品评分过,如果有不需要计算,如果没有 值为0,则进行计算
            if freq_matrix[user_id,item_id ] == 0:
                # 如果为0 计算推荐指数,
                recommend_index= cal_recommend_index_by_users(2, user_id, item_id ,freq_matrix,user_similar_matrix )
                # 算出来之后, 把这个数值保存到矩阵中去
                predict_matrix[user_id , item_id] = recommend_index
    return predict_matrix

In [47]:
# 调用上面的函数计算推荐系数矩阵
predict_matrix = cal_recommend_index_matrix(freq_matrix ,user_similar_matrix)
predict_matrix

user_id:  0
user_id:  1
user_id:  2
user_id:  3
user_id:  4
user_id:  5
user_id:  6
user_id:  7
user_id:  8
user_id:  9
user_id:  10
user_id:  11
user_id:  12
user_id:  13
user_id:  14
user_id:  15
user_id:  16
user_id:  17
user_id:  18
user_id:  19
user_id:  20
user_id:  21
user_id:  22
user_id:  23
user_id:  24
user_id:  25
user_id:  26
user_id:  27
user_id:  28
user_id:  29
user_id:  30
user_id:  31
user_id:  32
user_id:  33
user_id:  34
user_id:  35
user_id:  36
user_id:  37
user_id:  38
user_id:  39
user_id:  40
user_id:  41
user_id:  42
user_id:  43
user_id:  44
user_id:  45
user_id:  46
user_id:  47
user_id:  48
user_id:  49
user_id:  50
user_id:  51
user_id:  52
user_id:  53
user_id:  54
user_id:  55
user_id:  56
user_id:  57
user_id:  58
user_id:  59
user_id:  60
user_id:  61
user_id:  62
user_id:  63
user_id:  64
user_id:  65
user_id:  66
user_id:  67
user_id:  68
user_id:  69
user_id:  70
user_id:  71
user_id:  72
user_id:  73
user_id:  74
user_id:  75
user_id:  76
user_id: 

user_id:  596
user_id:  597
user_id:  598
user_id:  599
user_id:  600
user_id:  601
user_id:  602
user_id:  603
user_id:  604
user_id:  605
user_id:  606
user_id:  607
user_id:  608
user_id:  609
user_id:  610
user_id:  611
user_id:  612
user_id:  613
user_id:  614
user_id:  615
user_id:  616
user_id:  617
user_id:  618
user_id:  619
user_id:  620
user_id:  621
user_id:  622
user_id:  623
user_id:  624
user_id:  625
user_id:  626
user_id:  627
user_id:  628
user_id:  629
user_id:  630
user_id:  631
user_id:  632
user_id:  633
user_id:  634
user_id:  635
user_id:  636
user_id:  637
user_id:  638
user_id:  639
user_id:  640
user_id:  641
user_id:  642
user_id:  643
user_id:  644
user_id:  645
user_id:  646
user_id:  647
user_id:  648
user_id:  649
user_id:  650
user_id:  651
user_id:  652
user_id:  653
user_id:  654
user_id:  655
user_id:  656
user_id:  657
user_id:  658
user_id:  659
user_id:  660
user_id:  661
user_id:  662
user_id:  663
user_id:  664
user_id:  665
user_id:  666
user_i

array([[0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 2., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.],
       [4., 0., 3., ..., 0., 0., 0.]])

## 构建最终的推荐功能

1. 首先把上面的矩阵转换成DataFrame,方便操作
2. 现在的推荐矩阵不太方便操作,我们把数据转换回最初的那种格式
```
re_df_2 = re_df.stack().reset_index()
re_df_2
```
3. 现在我们需要根据每一个用户ID,提取推荐指数最大的两个产品
4. 所以我们需要对数据进行分组, 根据用户ID进行分组.
5. 如何提取出最大的两个数据呢? 默认没有这样功能, 所以我们自己构建一个函数,然后使用分组中的apply功能 , 应用这个聚合函数, 得到最终推荐结果.
6. 清理数据中的无效信息

In [26]:
df_pivot

产品ID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
用户ID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# 把这个推荐矩阵转换成DataFrame, 索引值使用之前的那个df_pivot的索引值
re_df = pd.DataFrame(predict_matrix ,index=df_pivot.index , 
             columns= df_pivot.columns)
re_df

产品ID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
用户ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,4.0,0.0,3.000000,0.000000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.0,0.0,0.0,0.0,0.0,0.0,4.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
# 把二维数据DataFrame 转化成 2重索引的 一维Series
re_df.stack()

用户ID  产品ID
1     1       0.0
      2       0.0
      3       0.0
      4       0.0
      5       0.0
             ... 
943   1678    0.0
      1679    0.0
      1680    0.0
      1681    0.0
      1682    0.0
Length: 1586126, dtype: float64

In [50]:
# 对多重索引的Series进行 reset_index , 就会把索引和列都变成DataFrame里面的数据
re_df_2 = re_df.stack().reset_index()
re_df_2

,用户ID,产品ID,0
0,1,1,0.0
1,1,2,0.0
2,1,3,0.0
3,1,4,0.0
4,1,5,0.0
...,...,...,...
1586121,943,1678,0.0
1586122,943,1679,0.0
1586123,943,1680,0.0
1586124,943,1681,0.0


In [51]:
# 把上面的列名0 改成 推荐指数
re_df_2.rename(columns={0:"推荐指数"} ,inplace=True)

In [52]:
re_df_2

,用户ID,产品ID,推荐指数
0,1,1,0.0
1,1,2,0.0
2,1,3,0.0
3,1,4,0.0
4,1,5,0.0
...,...,...,...
1586121,943,1678,0.0
1586122,943,1679,0.0
1586123,943,1680,0.0
1586124,943,1681,0.0


### 对上面的数据进行分组操作

以用户ID进行分组, 提取出每组中最大的两个推荐指数的数据

In [53]:
# 保存分组结果, 但是先不进行合并
re_df_grouped = re_df_2.groupby(by='用户ID')
re_df_grouped

In [54]:
# 定义函数功能, 参数n代表要提取几个东西
# 第一个参数分组结果
def get_topn(group, n):
    # group 就代表了每一组数据
    # 计算出每一组数据中的最大的两个数据的id 和指数
    # 首先对每组数据进行排序, 排序后切片出前两个最大的值
    r = group.sort_values(by='推荐指数', ascending=False)[:n]
    return r

### 用apply调用上面的函数进行分组操作

In [55]:
# 我们使用 分组中的apply功能, 这个功能可以自定义函数
topn = re_df_grouped.apply(get_topn, n = 5)
topn

用户ID  产品ID  推荐指数
用户ID                          
1    424         1   425   5.0
     332         1   333   5.0
     482         1   483   5.0
     650         1   651   5.0
     495         1   496   5.0
...            ...   ...   ...
943  1585193   943   750   5.0
     1584964   943   521   5.0
     1585462   943  1019   5.0
     1585367   943   924   5.0
     1584715   943   272   5.0

[4715 rows x 3 columns]

In [56]:
# 上面的结果多了一个无用的索引, 把它删除掉
# 把删除后的新的索引传递给这个表
topn.index = topn.index.droplevel(1)

In [58]:
topn.drop(columns='用户ID' , inplace =True)

In [59]:
# 得到最终的推荐结果
topn

,产品ID,推荐指数
用户ID,,
1,425,5.0
1,333,5.0
1,483,5.0
1,651,5.0
1,496,5.0
...,...,...
943,750,5.0
943,521,5.0
943,1019,5.0


### 或者是直接调用写好的函数

In [71]:
import recommend

In [72]:
import imp
imp.reload(recommend)


<module 'recommend' from '/Users/mac/FangCloudV2/个人文件/Python课件/pythoncode/备课文件夹/10.就业班三个案例研发/02.推荐系统案例研发/升达0925期推荐系统预习课件/recommend.py'>

In [73]:
from recommend import get_recom

In [74]:
get_recom(predict_matrix , df_pivot,k = 2)

,产品ID,推荐指数
用户ID,,
1,425,5.0
1,333,5.0
2,515,5.0
2,124,5.0
3,1612,5.0
...,...,...
941,50,5.0
942,333,5.0
942,8,5.0
